In [2]:
import requests
import json
import pandas as pd
import numpy as np
from datetime import datetime
import os
import glob
import time

First, you need to create a json file in the same directory as this notebook with the follow structure. 

```
{
    "APIKey":"put your Spoonacular API key here", 
}
```

In [3]:
with open("apikey.json") as f:
    content = f.read()
    API_KEY = json.loads(content)['aaltomail']

## Get recipe data from Spoonacular API

In [4]:
# Helper function
# export as csv
def export_df_as_csv(data):
    now = datetime.now()
    timestamp=f"{now.date()}-{now.hour}:{now.minute}:{now.second}"
    data.to_csv(f"data/recipedata-{timestamp}.csv")
    
def get_recipes(n, request_fn):
    '''
    This function calls request_fn n times to get n*100 recipes in total
    and create a DataFrame with the data
    
    return: pd.Dataframe
    '''
    recipes = []
    for _ in range(n):
        recipes_100, is_success = request_fn()
        if not is_success: 
            break
        recipes+=recipes_100
    df = pd.DataFrame(recipes)
    return df

### Random recipes

In [62]:
# number of recipes

n_hundred = 10 # 10 hundred = 1000 random recipes

In [63]:
def get_100_random_recipes():
    '''
    This function requests 100 random recipes from Spoonacular 
    100 is the highest number of recipes one could get from a single request
    
    return: 
        - List( Dict() ) - a list of recipes (full information)
        - True if status code is 200, False otherwise
    '''
    res = requests.get(f"https://api.spoonacular.com/recipes/random?apiKey={API_KEY}&number=100")
    
    if res.status_code == 200:
        recipes = json.loads(res.content)["recipes"]
        return recipes, True
    else:
        print(res.status_code)
        print(res.content)
        return [], False

In [64]:
# get data from spoonacular
recipe_data = get_recipes(n_hundred, get_100_random_recipes)

In [65]:
recipe_data.shape

(1000, 38)

In [66]:
export_df_as_csv(recipe_data)

### Get randome recipes based on complex search

One problem with the above approach is that it returns a very similar set of 1000 recipes for request. Therefore, less than 1000 recipes are collected even after multiple requests for 1000 recipes. 

To overcome this, we are going to use the complex search method, with the `sort` option set to `random`. 

In [5]:
def random_complex_search_100():
    '''
    This function gets 100 random recipes from Spoonacular with the complex search method
    
    returns: List( Dict() ) - a list of recipes (limited information)
    '''
    res = requests.get(f"https://api.spoonacular.com/recipes/complexSearch?apiKey={API_KEY}&number=100&sort=random")
    
    time.sleep(1)
    
    if res.status_code == 200:
        recipes = json.loads(res.content)["results"]
        return recipes, True
    else:
        print(res.status_code)
        print(res.content)
        return [], False

In [6]:
recipe_data1 = get_recipes(1, random_complex_search_100)

However, instead of getting all data related to the recipes, we only get `id`, `title`, `image` and `imageType`. To overcome this, we will get the information for each recipe by sending another set of requests. 

In [7]:
def get_recipe_info(recipe_ids):
    '''
    params:
        - recipes_ids: List(int) - list of recipe ids
    returns: 
        - pd.DataFrame
    '''
    recipes = []
    for recipe_id in recipe_ids:
        res = requests.get(f"https://api.spoonacular.com/recipes/{recipe_id}/information?apiKey={API_KEY}&includeNutrition=true")
        time.sleep(1)
        if res.status_code == 200:
            recipe = json.loads(res.content)
            recipes.append(recipe)
        else:
            print(res.status_code)
            print(res.content)
            break
    return pd.DataFrame(recipes)

In [8]:
recipe_ids = recipe_data1["id"].to_list()
recipe_info = get_recipe_info(recipe_ids)

402
b'{"status":"failure", "code":402,"message":"Your daily points limit of 150 has been reached. Please upgrade your plan to continue using the API."}'


In [9]:
recipe_info.head()

,vegetarian,vegan,glutenFree,dairyFree,veryHealthy,cheap,veryPopular,sustainable,weightWatcherSmartPoints,gaps,...,cuisines,dishTypes,diets,occasions,winePairing,instructions,analyzedInstructions,originalId,preparationMinutes,cookingMinutes
0,False,False,False,False,False,False,False,False,8,no,...,[],"[antipasti, starter, snack, appetizer, antipas...",[],[summer],{},Procedures ...,"[{'name': '', 'steps': [{'number': 1, 'step': ...",None,NaN,NaN
1,True,False,False,False,False,False,False,False,2,no,...,[],"[antipasti, starter, snack, appetizer, antipas...",[lacto ovo vegetarian],[],{},Directions ...,"[{'name': '', 'steps': [{'number': 1, 'step': ...",None,30.0,20.0
2,True,False,False,True,False,False,False,False,8,no,...,[],[side dish],"[dairy free, lacto ovo vegetarian]",[],{},None,[],None,25.0,75.0
3,False,False,False,False,False,False,False,False,5,no,...,[],"[lunch, main course, main dish, dinner]",[],[],{},Directions ...,"[{'name': '', 'steps': [{'number': 1, 'step': ...",None,15.0,15.0
4,False,False,False,True,False,False,False,False,29,no,...,[],"[lunch, main course, main dish, dinner]",[dairy free],[],"{'pairedWines': [], 'pairingText': '', 'produc...",None,[],None,60.0,15.0


In [10]:
print(f"{recipe_info.shape[0]} recipes downloaded")

33 recipes downloaded


In [11]:
export_df_as_csv(recipe_info)

## Grouping all csv files together

In [12]:
# This is just for us to know many recipes we originaly have 
grouped = pd.read_csv("./data/recipedata-grouped.csv")

In [13]:
# Group all csv files into one DataFrame

recipe_csv_files = glob.glob('./data/recipedata*.csv')
df = pd.concat(map(lambda path: pd.read_csv(path, index_col=0), recipe_csv_files))
df.head()

,vegetarian,vegan,glutenFree,dairyFree,veryHealthy,cheap,veryPopular,sustainable,weightWatcherSmartPoints,gaps,...,diets,occasions,winePairing,instructions,analyzedInstructions,originalId,spoonacularSourceUrl,preparationMinutes,cookingMinutes,author
0,True,False,False,False,False,False,False,False,9,no,...,['lacto ovo vegetarian'],[],"{'pairedWines': [], 'pairingText': '', 'produc...","<ol><li>You will need a 9"" springform pan, or ...","[{'name': '', 'steps': [{'number': 1, 'step': ...",NaN,https://spoonacular.com/orange-fig-teacake-wit...,NaN,NaN,NaN
1,True,False,False,False,False,False,False,False,21,no,...,['lacto ovo vegetarian'],[],{},<ol><li>In a frying pan heat up oil then add m...,"[{'name': '', 'steps': [{'number': 1, 'step': ...",NaN,https://spoonacular.com/poached-eggs-on-a-bed-...,NaN,NaN,NaN
2,True,False,False,True,False,False,False,False,5,no,...,"['dairy free', 'lacto ovo vegetarian']",[],{},<ol><li>Preheat the oven to 170C.</li><li>Blen...,"[{'name': '', 'steps': [{'number': 1, 'step': ...",NaN,https://spoonacular.com/pandan-chiffon-cake-65...,NaN,NaN,NaN
3,False,False,True,True,False,False,False,False,7,no,...,"['gluten free', 'dairy free', 'paleolithic', '...",[],"{'pairedWines': ['chardonnay', 'pinot noir', '...",<ol><li>Pre-heat your oven to 200C / 400F.</li...,"[{'name': '', 'steps': [{'number': 1, 'step': ...",NaN,https://spoonacular.com/pork-chop-with-honey-m...,NaN,NaN,NaN
4,False,False,False,False,False,False,False,False,19,no,...,[],"[""valentine's day"", ""father's day""]","{'pairedWines': ['chianti', 'trebbiano', 'verd...",Cooking beets\nHeat oven to 400 degrees\nWash ...,"[{'name': '', 'steps': [{'number': 1, 'step': ...",NaN,https://spoonacular.com/beet-gnocchi-with-stea...,NaN,NaN,NaN


In [14]:
print(f"Before dropping duplicates: {df.shape}")
df = df.drop_duplicates()
print(f"After dropping duplicates: {df.shape}")
print(f"{df.shape[0] - grouped.shape[0]} new recipes added.")

Before dropping duplicates: (5004, 40)
After dropping duplicates: (4438, 40)
33 new recipes added.


In [15]:
df.to_csv(f"data/recipedata-grouped.csv")